# Skeleton Code for the Final Project

Github repo link: https://github.com/GSDSAML/prompt-engineering-2024fall/tree/main



## Leaderboard

Leaderboard updated from the Github repo.

In [54]:
import requests

url = "https://raw.githubusercontent.com/GSDSAML/prompt-engineering-2024fall/refs/heads/main/final_project/leaderboard.csv"
response = requests.get(url)

with open("leaderboard.csv", "wb") as file:
    file.write(response.content)

print("File downloaded successfully.")


File downloaded successfully.


In [55]:
import pandas as pd

# Load the leaderboard.csv file into a pandas DataFrame.
df = pd.read_csv('leaderboard.csv')

df

,group_number,score,ranking,update_dt
0,0,0.2,1,2024-12-03
1,15,0.0,2,2024-12-03


## Data preparation

### You can download the FinQA train and test set here

In [46]:
!wget https://github.com/czyssrs/FinQA/raw/refs/heads/main/dataset/train.json

'wget' ���O�����Υ~���R�O�B�i���檺�{���Χ妸�ɡC


In [47]:
!wget https://github.com/czyssrs/FinQA/raw/refs/heads/main/dataset/test.json

'wget' ���O�����Υ~���R�O�B�i���檺�{���Χ妸�ɡC


## Environment Setup

In [3]:
# Install langchain
!pip install --quiet  langchain langchain-community langchainhub langchain-openai langchain-chroma bs4

In [49]:
!pip uninstall -y openai langchain langchain-openai
!pip install openai==0.27.8
!pip install langchain
!pip install langchain-openai

Found existing installation: openai 1.57.1
Uninstalling openai-1.57.1:
  Successfully uninstalled openai-1.57.1
Found existing installation: langchain 0.3.10
Uninstalling langchain-0.3.10:
  Successfully uninstalled langchain-0.3.10
Found existing installation: langchain-openai 0.2.12
Uninstalling langchain-openai-0.2.12:
  Successfully uninstalled langchain-openai-0.2.12
  Using cached openai-0.27.8-py3-none-any.whl.metadata (13 kB)
Using cached openai-0.27.8-py3-none-any.whl (73 kB)


   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 15.9 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.22
    Uninstalling langchain-core-0.3.22:
      Successfully uninstalled langchain-core-0.3.22


  Using cached langchain_openai-0.2.12-py3-none-any.whl.metadata (2.7 kB)
Using cached langchain_openai-0.2.12-py3-none-any.whl (50 kB)
  Attempting uninstall: openai
    Found existing installation: openai 0.27.8
    Uninstalling openai-0.27.8:
      Successfully uninstalled openai-0.27.8


### Set API key

### Prepare model

In [2]:
# Prepare model
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0, api_key=OPENAI_API_KEY)

## Data Preparation

### Download and unzip backup vector DB

*   Source dataset: [FinQA](https://finqasite.github.io/)
*   EDA for the FinQA dataset: https://medium.com/@sohilsharma1996/finqa-project-numerical-reasoning-over-financial-data-77a3fb4133d2

You can build your own vector DB for retrieval.

In [52]:
!wget https://github.com/GSDSAML/prompt-engineering-2024fall/raw/refs/heads/main/final_project/db.zip

'wget' ���O�����Υ~���R�O�B�i���檺�{���Χ妸�ɡC


In [53]:
!unzip db.zip

'unzip' ���O�����Υ~���R�O�B�i���檺�{���Χ妸�ɡC


### Fiscal year end date dictionary

Collected from SEC EDGAR

Dictionary with Key(ticker): Value(Fiscal year end date)

Actually, the date may different every year, but we will use this for now.

In [11]:
company_fiscal_year_end = {'GIS': '0530', 'FRT': '1231', 'MMM': '1231', 'K': '1228', 'INTC': '1228', 'CAG': '0525', 'MSI': '1231', 'FIS': '1231', 'CCI': '1231', 'CDW': '1231', 'GPN': '1231', 'TMUS': '1231', 'D': '1231', 'KMI': '1231', 'NTAP': '0426', 'TSCO': '1230', 'BLK': '1231', 'CNC': '1231', 'MRK': '1231', 'JKHY': '0630', 'VTR': '1231', 'GRMN': '1228', 'AMT': '1231', 'MAR': '1231', 'HIG': '1231', 'ADI': '1102', 'ILMN': '1229', 'NKE': '0531', 'VNO': '1231', 'FITB': '1231', 'ETR': '1231', 'KIM': '1231', 'AMAT': '1027', 'OKE': '1231', 'DG': '0131', 'TFX': '1231', 'AAL': '1231', 'EXPD': '1231', 'V': '0930', 'REGN': '1231', 'PPG': '1231', 'RCL': '1231', 'VLO': '1231', 'KHC': '1228', 'HUM': '1231', 'WELL': '1231', 'ALLE': '1231', 'UNP': '1231', 'LLY': '1231', 'TXN': '1231', 'GS': '1231', 'LKQ': '1231', 'SYY': '0629', 'IPG': '1231', 'AAP': '1228', 'ECL': '1231', 'MKTX': '1231', 'C': '1231', 'ZBH': '1231', 'UAA': '0331', 'MRO': '1231', 'AON': '1231', 'SPGI': '1231', 'ORLY': '1231', 'MO': '1231', 'PM': '1231', 'BKNG': '1231', 'BKR': '1231', 'AWK': '1231', 'APTV': '1231', 'WRK': '0930', 'SWKS': '0927', 'CMCSA': '1231', 'CE': '1231', 'CB': '1231', 'WMT': '0131', 'STT': '1231', 'RSG': '1231', 'AOS': '1231', 'AAPL': '0928', 'EXR': '1231', 'UA': '0331', 'CME': '1231', 'HII': '1231', 'HOLX': '0928', 'SLB': '1231', 'EW': '1231', 'EMR': '0930', 'VRTX': '1231', 'APD': '0930', 'ADBE': '1129', 'CAT': '1231', 'PNC': '1231', 'MAA': '1231', 'AES': '1231', 'SNA': '1228', 'JPM': '1231', 'NCLH': '1231', 'PKG': '1231', 'TROW': '1231', 'HST': '1231', 'NWS': '0630', 'MA': '1231', 'DVN': '1231', 'ANSS': '1231', 'ANET': '1231', 'SNPS': '1031', 'RL': '0328', 'DISH': '1231', 'HWM': '1231', 'FTV': '1231', 'L': '1231', 'LMT': '1231', 'MAS': '1231', 'STZ': '0228', 'EMN': '1231', 'CDNS': '1231', 'EL': '0630', 'SLG': '1231', 'BDX': '0930', 'UPS': '1231', 'MS': '1231', 'IQV': '1231', 'IP': '1231', 'EOG': '1231'}

## Define Tools

### LangChain Tool Use Example

List of pre-defined tools in LangChain: https://python.langchain.com/docs/integrations/tools/

In [4]:
from langchain_core.tools import tool


@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


# Let's inspect some of the attributes associated with the tool.
print(multiply.name)
print(multiply.description)
print(multiply.args)

@tool
def add(a: int, b: int) -> int:
    """Add two integers."""
    return a + b

print(add.name)
print(add.description)
print(add.args)

multiply
Multiply two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}
add
Add two integers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [5]:
tools = [add, multiply]

In [6]:
llm_with_tools = llm.bind_tools(tools)

query = "What is 3 * 12?"

llm_with_tools.invoke(query).tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 12},
  'id': 'call_RwY6KWnsW8cEtbfnLr6lMidZ',
  'type': 'tool_call'}]

In [7]:
from langchain_core.messages import HumanMessage

query = "What is 3 * 12? Also, what is 11 + 49?"

messages = [HumanMessage(query)]

ai_msg = llm_with_tools.invoke(messages)

print(ai_msg.tool_calls)

messages.append(ai_msg)

[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_oG2Zs41ZreHLYvaHzF2zZFfW', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'call_441xv8hYIqojX84kSCm7r59F', 'type': 'tool_call'}]


In [8]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_oG2Zs41ZreHLYvaHzF2zZFfW', 'function': {'arguments': '{"a": 3, "b": 12}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_441xv8hYIqojX84kSCm7r59F', 'function': {'arguments': '{"a": 11, "b": 49}', 'name': 'add'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 83, 'total_tokens': 133, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_6fc10e10eb', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-262f2e98-5785-4b79-89dc-d481851f02eb-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_oG2

In [9]:
llm_with_tools.invoke(messages).content

'The result of \\(3 \\times 12\\) is 36, and the result of \\(11 + 49\\) is 60.'

### Extract fiscal tool

In [12]:
company_fiscal_year_end

{'GIS': '0530',
 'FRT': '1231',
 'MMM': '1231',
 'K': '1228',
 'INTC': '1228',
 'CAG': '0525',
 'MSI': '1231',
 'FIS': '1231',
 'CCI': '1231',
 'CDW': '1231',
 'GPN': '1231',
 'TMUS': '1231',
 'D': '1231',
 'KMI': '1231',
 'NTAP': '0426',
 'TSCO': '1230',
 'BLK': '1231',
 'CNC': '1231',
 'MRK': '1231',
 'JKHY': '0630',
 'VTR': '1231',
 'GRMN': '1228',
 'AMT': '1231',
 'MAR': '1231',
 'HIG': '1231',
 'ADI': '1102',
 'ILMN': '1229',
 'NKE': '0531',
 'VNO': '1231',
 'FITB': '1231',
 'ETR': '1231',
 'KIM': '1231',
 'AMAT': '1027',
 'OKE': '1231',
 'DG': '0131',
 'TFX': '1231',
 'AAL': '1231',
 'EXPD': '1231',
 'V': '0930',
 'REGN': '1231',
 'PPG': '1231',
 'RCL': '1231',
 'VLO': '1231',
 'KHC': '1228',
 'HUM': '1231',
 'WELL': '1231',
 'ALLE': '1231',
 'UNP': '1231',
 'LLY': '1231',
 'TXN': '1231',
 'GS': '1231',
 'LKQ': '1231',
 'SYY': '0629',
 'IPG': '1231',
 'AAP': '1228',
 'ECL': '1231',
 'MKTX': '1231',
 'C': '1231',
 'ZBH': '1231',
 'UAA': '0331',
 'MRO': '1231',
 'AON': '1231',
 'SP

Extract company ticker from the question

We used LLM for now, but may need to connect DB for stability.

In [13]:
from langchain_core.prompts import PromptTemplate

extract_ticker_prompt = PromptTemplate(
    input_variables=["question"],
    template="""
    You have to extract the company name in the given question, and answer the ticker of the company.
    Answer in JSON format, with 'ticker' key.

    Question: {question}

    Output JSON: {{
      "ticker": "<ticker of the company>"
    }}
    """
)

from pydantic import BaseModel, Field

class Ticker(BaseModel):
    """Ticker of the company in the given question"""
    ticker: str = Field(description="ticker of the company")

extract_ticker_chain = extract_ticker_prompt | llm.with_structured_output(Ticker)

In [14]:
extract_ticker_chain.invoke({"question": "What was the revenue of Apple in 2022?"})

Ticker(ticker='AAPL')

In [15]:
from langchain_core.output_parsers import StrOutputParser
from datetime import datetime

parser = StrOutputParser()

change_prompt = PromptTemplate(
    input_variables=["original_question", "fiscal_info", "today"],
    template="""
    You have to convert relative time information in the given original question to the fiscal year of the company.
    Use the fiscal end date to convert to fiscal year.
    Answer only the converted question.

    Original question: {original_question}
    Fiscal end date for the company: {fiscal_info}
    Today date: {today}
    """
)

convert_chain = change_prompt | llm | parser

def get_fiscal_info_with_ticker(ticker: str) -> str:
    """Get fiscal year information of the company with ticker string."""
    ticker = ticker.upper()
    if ticker in company_fiscal_year_end:
      end_date = company_fiscal_year_end[ticker]
      return end_date[:2] + '-' + end_date[2:]
    else:
      return '12-31'

extract_fiscal_prompt = PromptTemplate(
    input_variables=["question"],
    template="""
    You have to extract the fiscal year in the given question.
    Answer in JSON format, with 'fy' key.

    Question: {question}

    Output JSON: {{
      "fy": <fiscal year in the question>
    }}
    """
)

from pydantic import BaseModel, Field

class FiscalYear(BaseModel):
    """FiscalYear mentioned in the given question"""
    fy: int = Field(description="fiscal year in the question")

extract_fiscal_chain = extract_fiscal_prompt | llm.with_structured_output(FiscalYear)

In [60]:
from langchain_core.tools import tool
from typing import Tuple, Optional
from pydantic import ValidationError

@tool
def rewrite2fiscal(question: str) -> Tuple[Optional[str], Optional[int]]:
    """
    Extract the company ticker and convert relative time information
    in the question to the fiscal year of the company.

    Args:
        question: The financial question from the user.

    Returns:
        A tuple of (ticker, fiscal year), or (None, None) if extraction fails.
    """
    try:
        # Extract ticker
        ticker_response = extract_ticker_chain.invoke({"question": question})
        ticker = ticker_response.ticker if hasattr(ticker_response, 'ticker') else None

        if not ticker:
            return None, None

        # Get today's date for fiscal year conversion
        today = datetime.now().strftime("%Y-%m-%d")
        fiscal_info = get_fiscal_info_with_ticker(ticker)

        # Convert question to fiscal year-specific query
        response = convert_chain.invoke({
            "original_question": question,
            "fiscal_info": fiscal_info,
            "today": today
        })

        # Extract fiscal year
        fiscal_response = extract_fiscal_chain.invoke({'question': response})
        fy = fiscal_response.fy if hasattr(fiscal_response, 'fy') else None

        return ticker, fy

    except Exception as e:
        print(f"Error in rewrite2fiscal: {e}")
        return None, None


### Domain specific calulation tools

You should implement more tools to answer complex questions.

In [19]:
@tool
def calculate_eps(net_income: float, outstanding_shares: int):
  """Calculate the EPS of the company using net income and outstanding share

   Args:
        net_income: Net income value of the company
        outstanding_shares: Total stock held by the company's shareholders

    Returns:
        EPS value
  """
  if (net_income is None) or (outstanding_shares is None):
    print("Give me the net income and outstanding shares first")
    return None
  elif outstanding_shares == 0:
    print("Outstanding shares cannot be zero")
    return None

  return net_income / outstanding_shares

In [20]:
@tool
def calculate_cashflowfromoperations(net_income: float, non_cash_items: float, changes_in_working_capital: float):
  """Calculate the cash flow from operations of the company using net income, non cash items and change in working capital

   Args:
        net_income: Net income value of the company
        non_cash_items: Financial transactions or events that are recorded in a company's financial statements but do not involve the exchange of cash
        changes_in_working_capital: Difference in a company's working capital between two reporting periods

    Returns:
        Value of cash flow from operations
  """
  if (net_income is None) or (non_cash_items is None) or (changes_in_working_capital is None):
    print("Give me the net income, non cash items and change in working capital data first")
    return None

  return net_income + non_cash_items + changes_in_working_capital

### our own tools

In [21]:
@tool
def calculate_operating_profit_margin(operating_profit: float, revenue: float):
    """Calculate the operating profit margin of the company.

    Args:
        operating_profit: Profit from business operations (gross profit less operating expenses) before deduction of interest and taxes.
        revenue: Revenue calculated from `calculate_revenue`.

    Returns:
        The operating profit margin, calculated as operating profit divided by revenue.
    """
    if (operating_profit is None) or (revenue is None) or (revenue == 0):
        print("Provide valid operating profit and revenue data first.")
        return None

    return operating_profit / revenue

@tool
def calculate_revenue(sales: float, average_price: float, sales_price: float, operating_revenues: dict):
    """
    Calculate the revenue of the company using sales, average price, and sales price.
    Additionally, calculate the average total operating revenue from the provided operating revenues for each year.

    Args:
        sales: Sales of the company.
        average_price: Average price of the service provided by the company.
        sales_price: Sales price of the company's product.
        operating_revenues: A dictionary where keys are years and values are total operating revenues for those years.

    Returns:
        A tuple containing:
        - Sales multiplied by average price.
        - Sales multiplied by sales price.
        - Average of total operating revenues for all years.
    """
    if (sales is None) or (average_price is None) or (sales_price is None):
        print("Provide sales, average price, and sales price data first.")
        return None

    # Calculate revenues based on sales and prices
    revenue_average_price = sales * average_price
    revenue_sales_price = sales * sales_price

    # Calculate the average of total operating revenues
    if operating_revenues:
        average_operating_revenue = sum(operating_revenues.values()) / len(operating_revenues)
    else:
        print("Provide a valid dictionary of operating revenues.")
        return None

    return revenue_average_price, revenue_sales_price, average_operating_revenue



In [22]:
@tool
def calculate_current_ratio(current_assets: float, current_liabilities: float):
    """Calculate the current ratio of a company using current assets and current liabilities.

    Args:
        current_assets: All company-owned assets that can be converted to cash within one year, includes cash, cash equivalents, accounts receivables, stock inventory, marketable securities, pre-paid liabilities, and other liquid assets.
        current_liabilities: Company's short-term financial obligations that are due within one year or within a normal operating cycle.

    Returns:
        The current ratio, calculated as current_assets / current_liabilities.
    """
    if (current_assets is None) or (current_liabilities is None):
        print("Provide both current assets and current liabilities data first")
        return None

    if current_liabilities == 0:
        print("Current liabilities cannot be zero to calculate the ratio")
        return None

    return current_assets / current_liabilities

@tool
def calculate_total_assets(total_liabilities: float, total_stockholders_equity: float):
    """Calculate the total assets of a company using total liabilities and total stockholders' equity.

    Args:
        total_liabilities: The combined debts that the company owes.
        total_stockholders_equity: The source of a company's assets, the owners' residual claim of a company's assets after its liabilities have been paid, or the company's total book value.

    Returns:
        The total assets, calculated as total_liabilities + total_stockholders_equity.
    """
    if (total_liabilities is None) or (total_stockholders_equity is None):
        print("Provide both total liabilities and total stockholders' equity data first")
        return None

    return total_liabilities + total_stockholders_equity

In [63]:
@tool
def calculate_ebt(net_income: float, taxes: float):
    """Calculate the EBT of the company using net income and taxes.

    Args:
        net_income: Sales minus cost of goods sold, general expenses, taxes, and interest.
        taxes: Total taxes paid by the company.

    Returns:
        EBT value (Net income + Taxes).
    """
    if (net_income is None) or (taxes is None):
        print("Provide both net income and taxes.")
        return None

    return net_income + taxes


@tool
def calculate_ebit(revenue: float, cogs: float, operating_expenses: float):
    """Calculate the EBIT of the company using revenue, COGS, and operating expenses.

    Args:
        revenue: Total revenue of the company.
        cogs: Cost of goods sold.
        operating_expenses: Total operating expenses of the company.

    Returns:
        EBIT value (Revenue – COGS – Operating Expenses).
    """
    if (revenue is None) or (cogs is None) or (operating_expenses is None):
        print("Provide revenue, COGS, and operating expenses.")
        return None

    return revenue - cogs - operating_expenses


@tool
def calculate_interest_expenses(calculate_ebit: float, calculate_ebt: float):
    """Calculate the interest expenses using EBIT and EBT.

    Args:
        calculate_ebit: EBIT value of the company.
        calculate_ebt: EBT value of the company.

    Returns:
        Interest expenses (EBIT - EBT).
    """
    if (calculate_ebit is None) or (calculate_ebt is None):
        print("Provide both EBIT and EBT.")
        return None

    return calculate_ebit - calculate_ebt


@tool
def calculate_interest_income(average_cash_and_cash_equivalents: float, cash_rate: float):
    """
    Calculate interest income using cash equivalents and interest rate.

    Args:
        average_cash_and_cash_equivalents: Average of the beginning and ending cash balances.
        cash_rate: Interest rate earned on cash (in percentage).

    Returns:
        Interest income (Average Cash * Cash Rate).
    """
    return average_cash_and_cash_equivalents * (cash_rate / 100)


@tool
def calculate_supplementary_leverage_ratio(liquid_assets: float, demand: float, time_liabilities: float):
    """Calculate the supplementary leverage ratio.

    Args:
        liquid_assets: Assets that can be converted to cash easily.
        demand: Consumer's desire to purchase goods/services.
        time_liabilities: Liabilities repayable after an agreed period.

    Returns:
        Supplementary leverage ratio in percentage.
    """
    if (liquid_assets is None) or (demand is None) or (time_liabilities is None):
        print("Provide liquid assets, demand, and time liabilities.")
        return None

    return (liquid_assets / (demand + time_liabilities)) * 100


@tool
def calculate_weighted_average(number: list, degree_of_importance: list, sum_of_number: float):
    """Calculate the weighted average.

    Args:
        number: A list of points in the dataset.
        degree_of_importance: Weights assigned to each number.
        sum_of_number: Sum of all numbers.

    Returns:
        Weighted average value.
    """
    if (number is None) or (degree_of_importance is None) or (sum_of_number is None):
        print("Provide numbers, degrees of importance, and their sum.")
        return None
    if len(number) != len(degree_of_importance):
        print("Numbers and degrees of importance must have the same length.")
        return None

    weighted_sum = sum(n * w for n, w in zip(number, degree_of_importance))
    return weighted_sum / sum_of_number


@tool
def calculate_minimum_lease_payment(pv: float, p: float, r: float, n: int, rv: float):
    """Calculate the minimum lease payment.

    Args:
        pv: Present value.
        p: Annual lease payments.
        r: Interest rate.
        n: Number of years in the lease term.
        rv: Residual value.

    Returns:
        Minimum lease payment.
    """
    if (pv is None) or (p is None) or (r is None) or (n is None) or (rv is None):
        print("Provide all required parameters (PV, P, r, n, RV).")
        return None

    total_lease_payment = sum(p / ((1 + r) ** i) for i in range(1, n + 1))
    residual_payment = rv / ((1 + r) ** n)
    return total_lease_payment + residual_payment


@tool
def calculate_cash_flow(total_income: float, total_expenses: float):
    """Calculate the cash flow of the company.

    Args:
        total_income: Total income of the company.
        total_expenses: Total expenses of the company.

    Returns:
        Cash flow (Total Income – Total Expenses).
    """
    if (total_income is None) or (total_expenses is None):
        print("Provide both total income and total expenses.")
        return None

    return total_income - total_expenses


### 1219 tools

In [43]:
from typing import Optional, List
@tool
def calculate_net_loss(gross_loss: float, gains: float):
    """Calculate the net loss given gross loss and gains."""
    if gross_loss is None or gains is None:
        return None
    return gross_loss - gains

@tool
def calculate_difference(value1: float, value2: float) -> float:
    """
    Calculate the difference between two numerical values.

    Args:
        value1: The first value.
        value2: The second value.

    Returns:
        The difference between value1 and value2.
    """
    return value1 - value2


@tool
def calculate_average(values: List[float]) -> float:
    """
    Calculate the average of a list of numerical values.

    Args:
        values: A list of numerical values.

    Returns:
        The average of the values.
    """
    return sum(values) / len(values)


@tool
def calculate_sum(values: List[float]) -> float:
    """
    Calculate the sum of a list of numerical values.

    Args:
        values: A list of numerical values.

    Returns:
        The sum of the values.
    """
    return sum(values)




### Retriever Tool

#### Build Vector DB

You can skip this part for demo.

In [72]:
import json

def load_and_view_sample(file_path):
  """Loads a JSON file and prints a sample of its contents."""
  try:
    with open(file_path, 'r') as f:
      data = json.load(f)
      print("Sample Data from JSON:")
      print(json.dumps(data, indent=2)[:500]) # Print first 500 chars for sample
  except FileNotFoundError:
    print(f"File not found: {file_path}")
  except json.JSONDecodeError:
    print(f"Error decoding JSON from file: {file_path}")


# Assuming your file is named 'test.json' in the current directory
load_and_view_sample('test.json')

Sample Data from JSON:
[
  {
    "pre_text": [
      "entergy corporation and subsidiaries management 2019s financial discussion and analysis a result of the entergy louisiana and entergy gulf states louisiana business combination , results of operations for 2015 also include two items that occurred in october 2015 : 1 ) a deferred tax asset and resulting net increase in tax basis of approximately $ 334 million and 2 ) a regulatory liability of $ 107 million ( $ 66 million net-of-tax ) as a result of customer credits 


In [73]:
import json

def count_rows_in_json(file_path):
  """Counts the number of rows (objects) in a JSON file."""
  try:
    with open(file_path, 'r') as f:
      data = json.load(f)
      if isinstance(data, list):
        return len(data)
      elif isinstance(data, dict):
        return len(data)
      else:
        return 0  # Or raise an error, depending on your needs
  except FileNotFoundError:
    print(f"File not found: {file_path}")
    return 0
  except json.JSONDecodeError:
    print(f"Error decoding JSON from file: {file_path}")
    return 0

# Assuming your file is named 'test.json' in the current directory
num_rows = count_rows_in_json('test.json')
print(f"Number of rows in the JSON file: {num_rows}")

Number of rows in the JSON file: 1147


In [74]:
import json

def analyze_json(file_path):
  """Analyzes a JSON file, printing information about its attributes."""
  try:
    with open(file_path, 'r') as f:
      data = json.load(f)
      if isinstance(data, list):
        print("JSON file contains a list of objects.")
        if len(data) > 0:
          print("Example object:")
          print(json.dumps(data[0], indent=2))
          for key in data[0].keys():
            print(f"- Attribute: {key}")
      elif isinstance(data, dict):
        print("JSON file contains a dictionary.")
        print("Attributes:")
        for key in data.keys():
          print(f"- Attribute: {key}")
      else:
        print("JSON file is empty or contains an unexpected data type.")
  except FileNotFoundError:
    print(f"File not found: {file_path}")
  except json.JSONDecodeError:
    print(f"Error decoding JSON from file: {file_path}")

# Analyze 'test.json'
analyze_json('test.json')

JSON file contains a list of objects.
Example object:
{
  "pre_text": [
    "entergy corporation and subsidiaries management 2019s financial discussion and analysis a result of the entergy louisiana and entergy gulf states louisiana business combination , results of operations for 2015 also include two items that occurred in october 2015 : 1 ) a deferred tax asset and resulting net increase in tax basis of approximately $ 334 million and 2 ) a regulatory liability of $ 107 million ( $ 66 million net-of-tax ) as a result of customer credits to be realized by electric customers of entergy louisiana , consistent with the terms of the stipulated settlement in the business combination proceeding .",
    "see note 2 to the financial statements for further discussion of the business combination and customer credits .",
    "results of operations for 2015 also include the sale in december 2015 of the 583 mw rhode island state energy center for a realized gain of $ 154 million ( $ 100 million n

In [75]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
from datetime import datetime
import json

# Define pre_text and post_text
pre_text = """
## Financial Data Analysis with Tools

This system is designed to analyze financial data and answer your questions about companies.
It uses tools like calculating metrics (EPS, cash flow from operations) and retrieving information from a financial dataset.

"""

post_text = """
**Note:** The system may not always have the information you're looking for.
If the response is insufficient, please refine your question or provide more details.
"""

# Define a table with examples
table = """
| Question Example | Expected Outcome |
|---|---|
| What was Apple's EPS in 2022? | Retrieves Apple's 2022 net income and shares outstanding, then calculates and returns EPS. |
| What was the cash flow from operations for Microsoft in 2023? | Retrieves Microsoft's net income, non-cash items, and changes in working capital, then calculates and returns cash flow from operations. |
"""

# Combine pre_text, post_text, and table
combined_text = f"{pre_text}\n\n**Example Usage:**\n\n{table}\n\n{post_text}"

# Function to ask ChatGPT and compare answers
def qa_and_compare(question):
    prompt_with_context = f"{combined_text}\n\n**Question:** {question}"

    messages = [HumanMessage(content=prompt_with_context)]

    ai_msg = llm.invoke(messages)

    print("ChatGPT's Answer:")
    print(ai_msg.content)


# Example question
question = "What was the revenue of Apple in 2022?"

qa_and_compare(question)

ChatGPT's Answer:
To provide you with the revenue of Apple in 2022, I would need to retrieve the relevant financial data. However, I currently do not have access to real-time financial datasets. 

As of my last update, Apple's revenue for the fiscal year 2022 was approximately $394.3 billion. For the most accurate and up-to-date information, please refer to Apple's official financial statements or a reliable financial news source.


In [203]:
with open('test.json', 'r') as f:
    train_data = json.load(f)

# Generate distinct_items_with_context
distinct_items_with_context = {}

for item in train_data:
    filename = item['filename']

    distinct_items_with_context[filename] = {
        'pre_text': item.get('pre_text', []),
        'post_text': item.get('post_text', []),
        'table': item.get('table', [])
    }

#Check the generated dictionary
print(f"Generated distinct_items_with_context with {len(distinct_items_with_context)} items.")

def merge_text_list(text_list):
    """Merge a list of strings into a single string with newline separation."""
    if not isinstance(text_list, list):
        raise ValueError("Input must be a list of strings.")
    return "\n".join(text_list)

Generated distinct_items_with_context with 380 items.


In [233]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from tqdm import tqdm  # Import tqdm for progress tracking

# Initialize text splitter and embeddings
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
embeddings = OpenAIEmbeddings(model='text-embedding-3-small', api_key=OPENAI_API_KEY)

# Function to process context data and save it into Chroma vector DB
def save_to_chroma(distinct_items_with_context, persist_directory="./db_test"):
    """
    Saves processed text data and metadata into a Chroma vector database.

    Args:
        distinct_items_with_context (dict): Dictionary containing context data with keys 'pre_text', 'post_text', and 'table'.
        persist_directory (str): Directory to save the vector database.
    """
    all_texts = []
    all_metadatas = []

    # Iterate through each file and process its context
    for filename, context_data in tqdm(distinct_items_with_context.items(), desc="Processing files"):
        # Extract ticker and fiscal year from filename or context_data (custom logic)
        ticker = filename.split('/')[0]  
        fiscal = filename.split('/')[1]  

        # Merge and split pre-text and post-text
        pre_text = merge_text_list(context_data.get('pre_text', []))
        post_text = merge_text_list(context_data.get('post_text', []))
        table = "\n".join([" | ".join(row) for row in context_data.get('table', [])])

        pre_text_chunks = splitter.split_text(pre_text)
        post_text_chunks = splitter.split_text(post_text)

        # Collect texts and corresponding metadata
        texts_for_db = pre_text_chunks + ([table] if table.strip() else []) + post_text_chunks
        metadatas_for_db = (
            [{'filename': filename, 'context_type': 'pre_text', 'company': ticker, 'fiscal': fiscal}] * len(pre_text_chunks) +
            ([{'filename': filename, 'context_type': 'table', 'company': ticker, 'fiscal': fiscal}] if table.strip() else []) +
            [{'filename': filename, 'context_type': 'post_text', 'company': ticker, 'fiscal': fiscal}] * len(post_text_chunks)
        )

        all_texts.extend(texts_for_db)
        all_metadatas.extend(metadatas_for_db)

    # Create and persist the vector database
    print(f"Saving {len(all_texts)} documents to Chroma DB...")
    docsearch = Chroma.from_texts(
        all_texts,
        embeddings,
        metadatas=all_metadatas,
        persist_directory=persist_directory
    )

    docsearch.persist()
    print("Chroma DB successfully persisted!")

# Example usage
# Assuming 'distinct_items_with_context' is already defined
save_to_chroma(distinct_items_with_context)


Processing files: 100%|██████████| 380/380 [00:00<00:00, 11280.12it/s]


Saving 4044 documents to Chroma DB...
Chroma DB successfully persisted!


#### Load Vector DB

In [78]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
import re
import json
from typing import Dict, List, Optional, Any

embeddings = OpenAIEmbeddings(model='text-embedding-3-small',api_key=OPENAI_API_KEY)

from langchain_chroma import Chroma

docsearch = Chroma(
    persist_directory="./test_db",
    embedding_function=embeddings
)

#### Retriever tool with filtered search

In [45]:
def retrieve(question:str, ticker: str, fy: int) -> list[str]:
  """Search vector DB for the financial reports with the question and ticker and fiscal year

   Args:
        question: Question need to be answered
        ticker: Ticker of the company for filtering the documents
        fy: Fiscal year for filtering the documents

    Returns:
        A related document for the question.
  """
  retriever = docsearch.as_retriever(search_kwargs={'k': 10, 'filter':
  {
      "$and": [
          {
              "company": {
                  "$eq": ticker
              }
          },
          {
              "fiscal": {
                  "$eq": fy
              }
          }
      ]
  }})

  retriever = docsearch.as_retriever(search_kwargs={"k": 1})
  result = retriever.invoke(question)
  print("Result without filters:", result)


  result = retriever.invoke(question)
  if result:
    return result
  else:
    return ["No data returned. Try again with correct ticker and fiscal year, or different question"]

In [46]:
# Check stored data
documents = docsearch.get()['documents']
metadatas = docsearch.get()['metadatas']

print(f"Number of documents in the DB: {len(documents)}")
print("Sample metadata:", metadatas[:5])  # Print metadata of the first 5 documents


Number of documents in the DB: 2089
Sample metadata: [{'company': 'ETR', 'context_type': 'pre_text', 'fiscal': 2016}, {'company': 'ETR', 'context_type': 'pre_text', 'fiscal': 2016}, {'company': 'ETR', 'context_type': 'pre_text', 'fiscal': 2016}, {'company': 'ETR', 'context_type': 'table', 'fiscal': 2016}, {'company': 'ETR', 'context_type': 'post_text', 'fiscal': 2016}]


In [47]:
retrieve("what was the hqla in the q4 of Citigroup in 2015?","C", 2015)

Result without filters: [Document(metadata={'company': 'C', 'context_type': 'table', 'fiscal': 2015}, page_content='in billions of dollars,dec . 31 2015,sept . 30 2015,dec . 31 2014\r\nhqla,$ 378.5,$ 398.9,$ 412.6\r\nnet outflows,336.5,355.6,368.6\r\nlcr,112% ( 112 % ),112% ( 112 % ),112% ( 112 % )\r\nhqla in excess of net outflows,$ 42.0,$ 43.3,$ 44.0\r\n')]


[Document(metadata={'company': 'C', 'context_type': 'table', 'fiscal': 2015}, page_content='in billions of dollars,dec . 31 2015,sept . 30 2015,dec . 31 2014\r\nhqla,$ 378.5,$ 398.9,$ 412.6\r\nnet outflows,336.5,355.6,368.6\r\nlcr,112% ( 112 % ),112% ( 112 % ),112% ( 112 % )\r\nhqla in excess of net outflows,$ 42.0,$ 43.3,$ 44.0\r\n')]

In [49]:
@tool
def retrieve_factual_data(question: str, ticker: str, fy: int) -> List[str]:
    """
    Retrieve relevant documents from the vector database based on the question, company ticker, and fiscal year.

    Args:
        question: The financial question from the user.
        ticker: The company ticker used to filter the documents.
        fy: The fiscal year used to filter the documents.

    Returns:
        A list of strings, where each string is the content of a relevant document.
        If no documents are found, returns a message indicating no data was found.
    """
    retriever = docsearch.as_retriever(search_kwargs={'k': 10})
    results = retriever.invoke(question)
    if not results:
        return ["No relevant documents found."]
    return [result.page_content for result in results]



def extract_key_terms(question: str) -> List[str]:
    """
    Extract key terms from the question to validate relevance.
    
    Args:
        question: The user's question.
    
    Returns:
        A list of key terms from the question.
    """
    terms = [word.strip(".,?") for word in question.split()]
    return [term for term in terms if len(term) > 2]  # Filter out short or non-informative terms


In [50]:
@tool
def retrieve_financial_data_tool(query: str) -> str:
    """
    Retrieve financial data for a given query, integrating tools for fiscal year and ticker extraction.

    Args:
        query: User's financial question.

    Returns:
        Relevant financial data or an informative message.
    """
    try:
        # Extract ticker and fiscal year
        ticker, fiscal_year = rewrite2fiscal(query)

        if not ticker or not fiscal_year:
            return "Error: Unable to determine company or fiscal year from the query."

        # Retrieve relevant financial data
        result = retrieve_factual_data.invoke({
            "question": query,
            "ticker": ticker,
            "fy": fiscal_year
        })

        if "Error" in result or "No data" in result:
            return (
                f"Could not find relevant data for {ticker} in fiscal year {fiscal_year}. "
                "Please refine your query or check the company name and fiscal year."
            )

        # Format result for better readability
        formatted_result = (
            # f"**Query:** {query}\n"
            # f"**Company (Ticker):** {ticker}\n"
            # f"**Fiscal Year:** {fiscal_year}\n"
            f"**Retrieved Data:**\n{result.strip()}"
        )

        return formatted_result

    except Exception as e:
        return f"Error during retrieval: {str(e)}"
    
def combine_documents(documents: List[str]) -> str:
    """
    Combine multiple documents into a single string for processing.
    """
    return "\n\n".join(documents)


In [71]:
from typing import Dict, List, Optional, Any

def extract_value_from_docs(documents: List[str], key: str) -> Optional[float]:
    """
    Extract a numerical value corresponding to a specific key from the documents.

    Args:
        documents: A list of retrieved documents.
        key: The key to search for in the documents.

    Returns:
        The extracted numerical value as a float, or None if the value is not found.
    """
    for doc in documents:
        if key.lower() in doc.lower():
            # Extract the numerical value (assuming it appears after the key)
            match = re.search(rf"{key}\s*[:\-]?\s*(\d+(\.\d+)?)", doc, re.IGNORECASE)
            if match:
                return float(match.group(1))
    return None

def extract_tool_inputs(tool_name: str, documents: List[str]) -> Dict[str, Any]:
    """
    Extract required inputs for a specific tool from the retrieved documents.

    Args:
        tool_name: The name of the tool.
        documents: A list of retrieved documents.

    Returns:
        A dictionary of extracted inputs for the tool, or an empty dictionary if no relevant data is found.
    """
    tool_inputs = {}

    # Example: Extract inputs for calculate_interest_income
    if tool_name == "calculate_interest_income":
        tool_inputs["average_cash_and_cash_equivalents"] = extract_value_from_docs(documents, "average cash")
        tool_inputs["cash_rate"] = extract_value_from_docs(documents, "interest rate")

    # Additional logic for other tools can be added here
    return tool_inputs

In [72]:
from langchain.tools import Tool
from langchain_core.messages import HumanMessage, SystemMessage

def calculate_financial_ratio_tool(company: str, year: str, ratio_type: str) -> str:  
    try:  
        if ratio_type == "current_ratio":  
            current_assets = retrieve_financial_data_tool(f"current assets {company} {year}")  
            current_liabilities = retrieve_financial_data_tool(f"current liabilities {company} {year}")  
            return float(current_assets) / float(current_liabilities)  
        elif ratio_type == "operating_margin":  
            operating_income = retrieve_financial_data_tool(f"operating income {company} {year}")  
            revenue = retrieve_financial_data_tool(f"revenue {company} {year}")  
            return (float(operating_income) / float(revenue)) * 100  
    except Exception as e:  
        return f"Error calculating ratio: {str(e)}"  

def perform_math_operations_tool(operation: str, values: list) -> str:  
    try:  
        values = [float(v) for v in values]  
        if operation == "difference":  
            return values[0] - values[1]  
        elif operation == "average":  
            return sum(values) / len(values)  
        elif operation == "sum":  
            return sum(values)  
        elif operation == "multiply":  
            result = 1  
            for v in values:  
                result *= v  
            return result  
    except Exception as e:  
        return f"Error in calculation: {str(e)}"  
tools = [
    Tool(
        name="retrieve_financial_data",
        func=retrieve_financial_data_tool,
        description="Retrieve basic financial data for a company."
    ),
    Tool(
        name="calculate_financial_ratio",
        func=calculate_financial_ratio_tool,
        description="Calculate financial ratios like current ratio or operating margin."
    ),
    Tool(
        name="perform_math_operations",
        func=perform_math_operations_tool,
        description="Perform mathematical operations on financial data."
    ),
    Tool(
        name="calculate_operating_profit_margin",
        func=calculate_operating_profit_margin,
        description="Calculate the operating profit margin of the company."
    ),
    Tool(
        name="calculate_revenue",
        func=calculate_revenue,
        description="Calculate revenue based on sales data and average operating revenues."
    ),
    Tool(
        name="calculate_ebt",
        func=calculate_ebt,
        description="Calculate the EBT (Earnings Before Taxes) of the company."
    ),
    Tool(
        name="calculate_ebit",
        func=calculate_ebit,
        description="Calculate the EBIT (Earnings Before Interest and Taxes) of the company."
    ),
    Tool(
        name="calculate_interest_expenses",
        func=calculate_interest_expenses,
        description="Calculate the interest expenses using EBIT and EBT."
    ),
    Tool(
        name="calculate_interest_income",
        func=calculate_interest_income,
        description="Calculate interest income based on average cash equivalents and interest rate."
    ),
    Tool(
        name="calculate_supplementary_leverage_ratio",
        func=calculate_supplementary_leverage_ratio,
        description="Calculate the supplementary leverage ratio of the company."
    ),
    Tool(
        name="calculate_weighted_average",
        func=calculate_weighted_average,
        description="Calculate the weighted average of a dataset based on weights."
    ),
    Tool(
        name="calculate_minimum_lease_payment",
        func=calculate_minimum_lease_payment,
        description="Calculate the minimum lease payment considering lease terms and residual value."
    ),
    Tool(
        name="calculate_cash_flow",
        func=calculate_cash_flow,
        description="Calculate the cash flow of the company based on total income and expenses."
    ),
    Tool(
        name="calculate_net_loss",
        func=calculate_net_loss,
        description="Calculate the net loss given gross loss and gains."
    ),
    Tool(
        name="calculate_difference",
        func=calculate_difference,
        description="Calculate the difference between two numerical values."
    ),
    Tool(
        name="calculate_average",
        func=calculate_average,
        description="Calculate the average of a list of numerical values."
    ),
    Tool(
        name="calculate_sum",
        func=calculate_sum,
        description="Calculate the sum of a list of numerical values."
    ),
    Tool(
        name="extract_key_terms",
        func=extract_key_terms,
        description="Extract key terms from a question to determine relevant tools."
    )
]



In [73]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

response_prompt = PromptTemplate(
    input_variables=["query", "data"],
    template="""
    You are a financial analysis expert. Your goal is to provide structured answers to financial questions.

    Question: {query}
    Relevant Data: {data}

    Response Format:
    - Answer: [Provide a specific numerical value or clear conclusion.]
    - Reasoning: [Explain how the answer was derived, including calculations or decisions made.]
    - Tools Used: [List the tools used, their inputs, and outputs.]
    """
)

# Function to generate a response
def generate_response(query: str, data: str) -> str:
    """
    Generates a structured response to a financial query using LLM.

    Args:
        query (str): The financial question.
        data (str): Relevant data for answering the query.

    Returns:
        str: A structured response including an answer and reasoning.
    """
    # Format the prompt
    prompt = response_prompt.format(query=query, data=data)

    # Invoke the LLM directly
    response = llm(prompt)

    return response


## Agent with tools

In [74]:
def tool_selector(keywords: List[str]) -> List[str]:
    """
    Map keywords to relevant tools.
    """
    tool_mapping = {
        "current ratio": ["calculate_current_ratio"],
        "operating profit margin": ["calculate_operating_profit_margin"],
        "revenue": ["retrieve_financial_data_tool", "calculate_revenue"],
        "cashflow": ["retrieve_financial_data_tool"],
        "lease payment": ["calculate_minimum_lease_payment"],
        "interest": ["calculate_interest_expenses", "calculate_interest_income"],
    }
    selected_tools = []
    for keyword in keywords:
        selected_tools.extend(tool_mapping.get(keyword, []))
    return list(set(selected_tools))  # Remove duplicates

def agentic_rag(query: str) -> str:
    # Step 1: Extract keywords
    keywords = extract_key_terms(query)  

    # Step 2: Select relevant tools
    tools_to_use = tool_selector(keywords)

    # Step 3: Extract ticker and fiscal year
    ticker, fy = rewrite2fiscal(query)  
    if not ticker or not fy:
        return "Error: Unable to determine company or fiscal year from the query."

    # Step 4: Retrieve documents and combine them
    documents = retrieve_factual_data.invoke({"question": query, "ticker": ticker, "fy": fy})
    combined_data = combine_documents(documents)

    # Step 5: Use selected tools (if any) to enrich data
    tool_responses = []
    for tool in tools_to_use:
        if tool in globals():
            # Extract the required parameters for the tool
            tool_inputs = extract_tool_inputs(tool, documents)

            if tool_inputs:  # Ensure that parameters are available
                try:
                    tool_response = globals()[tool](**tool_inputs)
                    tool_responses.append(tool_response)
                except Exception as e:
                    print(f"Error calling tool {tool}: {e}")
            else:
                print(f"No inputs found for tool {tool}. Skipping...")


    # Step 6: Generate structured response
    enriched_data = combined_data + "\n\n" + "\n\n".join(tool_responses)
    structured_response = generate_response(query, enriched_data)
    return structured_response

## Scoring section

Use this Q&A dictionary to test your project code.

You should not change the scoring prompt.

You can submit your accuracy result with the screenshot to TA email, then I will post the score to the leaderboard.

In [54]:
qa_dict = [{'Question': '"what was the hqla in the q4 of Citigroup in 2015"',
  'Answer': '"378.5 billion dollars"'},
 {'Question': '"as a result of the sales of certain non-core towers and other assets what was the recorded net losses of American Tower in 2008"',
  'Answer': '"10.5 million dollars"'},
 {'Question': '"what was the port call costsin of Royal Caribbean Cruises in 2012"',
  'Answer': '"459.8 million dollars"'},
 {'Question': '"what is the aggregate rent expense of American Tower Corp in 2014?"',
  'Answer': '"655.0 million dollars"'},
 {'Question': '"what is the long-term component of BlackRock at 12/31/2011?"',
  'Answer': '"593356 million dollars"'},
 {'Question': '"at december 31 , 2012 of The PNC Financial Services, what was the potential maximum exposure under the loss share arrangements?"',
  'Answer': '"3.9 billion dollars"'},
 {'Question': '"what is the total financial liabilities at fair value of Goldman Sachs in 2012?"',
  'Answer': '"377677 million dollars"'},
 {'Question': '"what was brazilian paper sales of International Paper Company in 2006?"',
  'Answer': '"496 million dollars"'},
 {'Question': '"what was brazilian paper sales of International Paper Company in 2005?"',
  'Answer': '"465 million dollars"'},
 {'Question': '"what is the cashflow of Kellogg in 2006?"',
  'Answer': '"957.4 million dollars"'},
 {'Question': '"From the perspective of 5 years ago, what percentage of total minimum lease payments of Dish Network are due in 2015?"',
  'Answer': '"9.772%"'},
 {'Question': '"in 2011 what was the SL Green Realty Corp\'s percent of the change in the account balance at end of year"',
  'Answer': '"6.75%"'},
 {'Question': '"what was the value in thousands of unvested restricted stock and performance awards at the weighted-averagegrant-datefair value as of december 31 , 2018 of Global Payments?"',
  'Answer': '"117624.84"'},
 {'Question': '"based on the cash dividends paid of the year, how many common stock shares were outstanding in 2007 Snap-on?"',
  'Answer': '"58909091"'},
 {'Question': '"what is the Entergy Corp\'s 2008 total value , in millions of dollars , of issuable long-term securities?"',
  'Answer': '"1450"'},
 {'Question': '"for the capital framework of Goldman Sachs Group 7 years ago, what percent of the minimum supplementary leverage ratio consisted of a buffer?"',
  'Answer': '"40%"'},
 {'Question': '"in 2007 what was the 3M\'s ratio of the interest expense to the interest income"',
  'Answer': '"1.59"'},
 {'Question': '"what is the AON\'s decrease observed in the additions for tax positions of prior years as of 2018, in millions?"',
  'Answer': '"2"'},
 {'Question': '"In 2014 Global Payments, what is the total value of securities approved by security holders but net yer issued , ( in millions ) ?"',
  'Answer': '"365.4"'},
 {'Question': '"what is the percent of our network route miles that is owned rather than operated on pursuant to trackage rights or leases in 2016 Union Pacific Corp"',
  'Answer': '"81.2%"'},
 {'Question': '"What is the American Tower Corp\'s current ratio in 12 years ago?"',
  'Answer': 1.7923},
 {'Question': '"What is the DISH Network Corporation\'s current ratio in 13 years ago?"',
  'Answer': 1.7804},
 {'Question': '"What is the American Tower Corp\'s current ratio in 2012?"',
  'Answer': 1.7923},
 {'Question': '"What is the DISH Network Corporation\'s current ratio in 2011?"',
  'Answer': 1.7804},
 {'Question': '"What is the International Paper Company\'s Operating Profit Margin in 2006?"',
  'Answer': 9.769119769},
 {'Question': '"What is the International Paper Company\'s Operating Profit Margin in 2005?"',
  'Answer': 6.59693166},
 {'Question': '"What is the International Paper Company\'s Operating Profit Margin in 2004?"',
  'Answer': 7.119831815},
 {'Question': '"What is the Air Products and Chemicals\'s Operating Profit Margin in 2016?"',
  'Answer': 26.7735375},
 {'Question': '"What is the Air Products and Chemicals\'s Operating Profit Margin in 2015?"',
  'Answer': 21.88472888},
 {'Question': '"What is the Air Products and Chemicals\'s Operating Profit Margin in 2014?"',
  'Answer': 18.69805075},
 {'Question': '"What is the difference of Operating Profit Margin between Air Products and Chemicals in 2016 and International Paper Company in 2006?"',
  'Answer': '17.00441774'},
 {'Question': '"What is the average of Operating Profit Margin between Air Products and Chemicals in 2016 and International Paper Company in 2005?"',
  'Answer': '16.68523458'},
 {'Question': '"What is the sum of Operating Profit Margin between Air Products and Chemicals in 2016 and International Paper Company in 2004?"',
  'Answer': '33.89336932'},
 {'Question': '"Which one\'s Operating Profit Margin is bigger between Air Products and Chemicals in 2014 and International Paper Company in 2006?"',
  'Answer': 'Air Products and Chemicals'},
 {'Question': '"What is the difference of Operating Profit Margin between Air Products and Chemicals in 2015 and International Paper Company in 2006?"',
  'Answer': '12.11560911'},
 {'Question': '"What is the average of Operating Profit Margin between Air Products and Chemicals in 2015 and International Paper Company in 2005?"',
  'Answer': '14.24083027'},
 {'Question': '"What is the sum of Operating Profit Margin between Air Products and Chemicals in 2015 and International Paper Company in 2004?"',
  'Answer': '29.00456069'},
 {'Question': '"Which one\'s current ratio is bigger between American Tower Corp in 2012 and DISH Network Corporation in 2011?"',
  'Answer': 'American Tower Corp'},
 {'Question': '"What is the average of current ratio between American Tower Corp in 2012 and DISH Network Corporation in 2011?"',
  'Answer': '1.786341177'},
 {'Question': '"What is the difference of current ratio between American Tower Corp in 2012 and DISH Network Corporation in a year before that?"',
  'Answer': '0.01185521735'},
 {'Question': '"What is the Goldman Sachs Group\'s Total Assets change from 2014 to 2016 multiplied by the average of current ratio between American Tower Corp in 2012 and DISH Network Corporation in 2011?"',
  'Answer': '48722.45 million'},
 {'Question': '"What is the Goldman Sachs Group\'s Total Assets change from 2013 to 2017 multiplied by the average of current ratio between American Tower Corp in 2012 and DISH Network Corporation in 2011?"',
  'Answer': '46296.6 million'},
 {'Question': '"What is the sum of Goldman Sachs Group\'s Total Assets from 2011 to 2014 divided by the average of Operating Profit Margin between Air Products and Chemicals in 2016 and International Paper Company in 2005?',
  'Answer': '217529.28 million'},
 {'Question': '"What is the sum of Goldman Sachs Group\'s Total Assets from 2011 to 2014 multiplied by the average of Operating Profit Margin between Air Products and Chemicals in 2015 and International Paper Company in 2005?"',
  'Answer': '51687477.96 million'},
 {'Question': '"What is the Goldman Sachs Group\'s Total Assets change from 2014 to 2016 divided the difference of Operating Profit Margin between Air Products and Chemicals in 2016 and International Paper Company in 2006?"',
  'Answer': '1604 million'},
 {'Question': '"What is the Goldman Sachs Group\'s Total Assets change from 2013 to 2017 dividied the sum of Operating Profit Margin between Air Products and Chemicals in 2016 and International Paper Company in 2004?"',
  'Answer': '764.6 million'},
 {'Question': '"What is the difference of Union Pacific Corp\'s revenue from 2013 to 2016 divided the sum of Operating Profit Margin between Air Products and Chemicals in 2015 and International Paper Company in 2004?"',
  'Answer': '-69.7 million'},
 {'Question': '"What is the average of Union Pacific Corp\'s revenue from 2013 to 2016 divided by the sum of Operating Profit Margin between Air Products and Chemicals in 2015 and International Paper Company in 2004?"',
  'Answer': '755.95 million'},
 {'Question': '"What is the difference of Union Pacific Corp\'s revenue from 2013 to 2016 multiplied by the average of current ratio between American Tower Corp in 2012 and DISH Network Corporation in 2011?"',
  'Answer': '-3611.98 million'},
 {'Question': '"What is the average of Union Pacific Corp\'s revenue from 2013 to 2016 divided by the average of Operating Profit Margin between Air Products and Chemicals in 2015 and International Paper Company in 2005?"',
  'Answer': '1539.67 million'}]

In [55]:
score_answer_prompt = PromptTemplate(
    input_variables=["question", "answer", "response"],
    template="""
    You have to score the response by comparing with the answer.
    You should score 0 or 1 as JSON with "score" key.
    You can ignore minor difference with the unit or numerical value.
    Question: {question}
    Answer: {answer}
    Response: {response}
    Score:

    Output JSON: {{
      "score": 1 if the answer response is meaningfully the same as the answer, 0 if the response is different from the answer>
    }}
    """
)

from pydantic import BaseModel, Field

class Score(BaseModel):
    """Score of the response"""
    score: int = Field(description="score of the response")

score_answer_chain = score_answer_prompt | llm.with_structured_output(Score)

In [56]:
score_answer_chain.invoke({'question': qa_dict[0]['Question'], 'answer': qa_dict[0]['Answer'], 'response': 'it is 375 B'}).score

1

### Accuracy score

In [79]:
# Initialize variables
results = []
correct = 0

# Iterate over all questions in the dataset
for i, item in enumerate(qa_dict):
    # Generate model's response
    response = agentic_rag(item['Question'])

    # Extract content if the response has a 'content' attribute
    if hasattr(response, 'content'):
        response = response.content
    print(response)
    
    # Compute the score for the response
    score = score_answer_chain.invoke({
        'question': item['Question'],
        'answer': item['Answer'],
        'response': response
    }).score
    correct += score

    # Record the result for each question
    result = {
        "index": i,  
        "question": item['Question'],  
        "model_answer": response,  
        "correct_answer": item['Answer'],  
        "score": score  
    }
    results.append(result)

    print(i, correct)

# Calculate overall accuracy
accuracy = correct / len(qa_dict)
print(f"Accuracy: {correct}/{len(qa_dict)}")

def serialize_results(results):
    """
    Convert non-JSON serializable objects into serializable ones.
    """
    if isinstance(results, list):
        return [serialize_results(item) for item in results]
    elif isinstance(results, dict):
        return {key: serialize_results(value) for key, value in results.items()}
    elif hasattr(results, 'content'):  
        return results.content
    else:
        return results  

serializable_results = serialize_results(results)

output_file = "1219results.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(serializable_results, f, ensure_ascii=False, indent=4)

print(f"Results saved to {output_file}")


- Answer: The High-Quality Liquid Assets (HQLA) for Citigroup in Q4 of 2015 was $378.5 billion.

- Reasoning: The relevant data provided indicates that the HQLA for Citigroup as of December 31, 2015, was $378.5 billion. This figure is directly taken from the data set provided, which lists HQLA values for different dates, including September 30, 2015, and December 31, 2014, for comparison.

- Tools Used: 
  - Data extraction from the provided financial data table.
  - No calculations were necessary as the answer was directly available from the data.
0 1
- Answer: The recorded net losses of American Tower in 2008 due to the sales of certain non-core towers and other assets were approximately $10.5 million.

- Reasoning: The relevant data indicates that during the year ended December 31, 2008, American Tower recorded net losses from asset sales and other impairments totaling $10.7 million, which was offset by gains from asset sales of $0.2 million. Therefore, the net loss attributable to 

In [38]:
test_question = "what was the hqla in the q4 of Citigroup in 2015"
response = agentic_rag(test_question)
print("Generated Response:", response)


Generated Response: content="Answer: The specific numerical value for Citigroup's High-Quality Liquid Assets (HQLA) in Q4 of 2015 is not provided in the retrieved data.\n\nReasoning: The retrieved data discusses Citigroup's liquidity measures and mentions the Liquidity Coverage Ratio (LCR) and the Net Stable Funding Ratio (NSFR), but it does not provide specific figures for HQLA in Q4 2015. To answer the question accurately, I would need the actual HQLA figure from Citigroup's financial statements or regulatory filings for that quarter. \n\nTo derive the HQLA, one would typically look at the balance sheet or liquidity disclosures in Citigroup's Q4 2015 earnings report or 10-K filing, which would detail the amount of HQLA held at that time. Since this information is not available in the provided data, I cannot calculate or estimate the HQLA without additional data.\n\nTools Used: \n- Financial statements (not available in the retrieved data)\n- Regulatory filings (not available in the r

In [39]:
query = "as a result of the sales of certain non-core towers and other assets what was the recorded net losses of American Tower in 2008"
retrieved_docs = docsearch.similarity_search(query, k=10)

for idx, doc in enumerate(retrieved_docs):
    print(f"Document {idx+1}:", doc.page_content)


Document 1: american tower corporation and subsidiaries notes to consolidated financial statements 2014 ( continued ) 12 .impairments , net loss on sale of long-lived assets , restructuring and merger related expense the significant components reflected in impairments , net loss on sale of long-lived assets , restructuring and merger related expense in the accompanying consolidated statements of operations include the following : impairments and net loss on sale of long-lived assets 2014during the years ended december 31 , 2005 , 2004 and 2003 , the company recorded impairments and net loss on sale of long-lived assets ( primarily related to its rental and management segment ) of $ 19.1 million , $ 22.3 million and $ 28.3 million , respectively .2022 non-core asset impairment charges 2014during the years ended december 31 , 2005 and 2004 respectively , the company sold a limited number of non-core towers and other non-core assets and recorded impairment charges to write-down these and 

In [155]:
response = agentic_rag("as a result of the sales of certain non-core towers and other assets what was the recorded net losses of American Tower in 2008")
print(response.messages)
    
# Compute the score for the response
score = score_answer_chain.invoke({    
    'question': item['Question'],
    'answer': "10.5 million dollars",
    'response': response
}).score
print(score)

AttributeError: 'str' object has no attribute 'messages'